# Spaceship Titanic Competition

Authors: Jimmy Bierenbroodspot, Maarten de Jue

In [ ]:
# Data manipulation
import pandas as pd
import numpy  as np

# Visualization
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

# Table of Contents

- [Business Understanding](#Business-Understanding)
- [Data Understanding](#Data-Understanding)
- [Data Preparation](#Data-Preparation)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Modeling](#Modeling)

# Business Understanding

[To table of contents](#Table-of-Contents)

# Data Understanding

[To table of contents](#Table-of-Contents)

# Data Preparation

[To table of contents](#Table-of-Contents)

# Exploratory Data Analysis

[To table of contents](#Table-of-Contents)

# Modeling

[To table of contents](#Table-of-Contents)